In [1]:

import cv2
import numpy as np
import keras
import os
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, Conv2DTranspose, BatchNormalization, Conv2D, Activation, add
from keras.layers import GlobalMaxPooling2D, Flatten, PReLU, Lambda
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

from model import SRCNN, FSRCNN, FSRCNN2, SRCNNex, PSNR, generator_SR
from keras.optimizers import Nadam, Adam, SGD

Using TensorFlow backend.


In [2]:
with open("train.txt", "r") as f :
    train = [line.rstrip() for line in f]
    
with open("Set5.txt", "r") as f :
    test = [line.rstrip() for line in f]
    

K.clear_session()
scales = 2
name_model = "FSRCNN3.hdf5"
checkpoint = ModelCheckpoint("weights/"+name_model, verbose=2, monitor='val_PSNR', save_best_only=True, save_weights_only=False, mode='max')

opt = Nadam(0.001) #SGD(0.001, nesterov=True, decay=0.0005, momentum=0.9)#
     
model = FSRCNN(scale=scales)
model.compile(loss="mse", optimizer=opt, metrics=[PSNR])
model.summary()
train_gen = generator_SR(train, scales, "FSRCNN")
steps_train = train_gen.getSteps()

test_gen = generator_SR(test, scales, "FSRCNN")
steps_test = test_gen.getSteps()

lr_decay = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, 
                                 verbose=1, mode='min', epsilon=0.00005, cooldown=0, min_lr=0.0000001)    


model.fit_generator(train_gen.generator(True, True), steps_per_epoch=steps_train, 
                    validation_data=test_gen.generator(), validation_steps=steps_test, 
                    epochs=500, verbose=1, 
                    callbacks=[lr_decay, checkpoint])



Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, None, None, 3)     0         
_________________________________________________________________
lambda_1 (Lambda)            (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 56)    4256      
_________________________________________________________________
conv2 (Conv2D)               (None, None, None, 12)    684       
_________________________________________________________________
conv3_0 (Conv2D)             (None, None, None, 12)    1308      
_________________________________________________________________
conv3_1 (Conv2D)             (None, None, None, 12)    1308      
_________________________________________________________________
conv

C:\Users\Shiro\Anaconda3\envs\DL\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/500
700/700 [==============================] - 134s 192ms/step - loss: 1315.4776 - PSNR: 18.4950 - val_loss: 301.6731 - val_PSNR: 23.5746

Epoch 00001: val_PSNR improved from -inf to 23.57457, saving model to weights/FSRCNN3.hdf5
Epoch 2/500
700/700 [==============================] - 110s 157ms/step - loss: 435.3648 - PSNR: 22.8795 - val_loss: 180.0340 - val_PSNR: 25.8662

Epoch 00002: val_PSNR improved from 23.57457 to 25.86621, saving model to weights/FSRCNN3.hdf5
Epoch 3/500
700/700 [==============================] - 99s 141ms/step - loss: 331.3752 - PSNR: 24.1766 - val_loss: 146.8509 - val_PSNR: 26.6494

Epoch 00003: val_PSNR improved from 25.86621 to 26.64940, saving model to weights/FSRCNN3.hdf5
Epoch 4/500
700/700 [==============================] - 99s 142ms/step - loss: 240.3683 - PSNR: 25.4396 - val_loss: 116.5728 - val_PSNR: 27.8046

Epoch 00004: val_PSNR improved from 26.64940 to 27.80458, saving model to weights/FSRCNN3.hdf5
Epoch 5/500
700/700 [====================

700/700 [==============================] - 70s 101ms/step - loss: 98.1486 - PSNR: 29.8811 - val_loss: 50.0071 - val_PSNR: 31.8575

Epoch 00039: val_PSNR improved from 31.75571 to 31.85753, saving model to weights/FSRCNN3.hdf5
Epoch 40/500
700/700 [==============================] - 66s 94ms/step - loss: 113.8932 - PSNR: 29.5593 - val_loss: 62.3269 - val_PSNR: 30.5856

Epoch 00040: val_PSNR did not improve from 31.85753
Epoch 41/500
700/700 [==============================] - 72s 102ms/step - loss: 101.0521 - PSNR: 29.8233 - val_loss: 52.0834 - val_PSNR: 31.6348

Epoch 00041: val_PSNR did not improve from 31.85753
Epoch 42/500
700/700 [==============================] - 68s 98ms/step - loss: 108.3356 - PSNR: 29.7288 - val_loss: 56.6765 - val_PSNR: 31.2651

Epoch 00042: val_PSNR did not improve from 31.85753
Epoch 43/500
700/700 [==============================] - 67s 96ms/step - loss: 95.4111 - PSNR: 30.0146 - val_loss: 50.4969 - val_PSNR: 31.7984

Epoch 00043: val_PSNR did not improve from

700/700 [==============================] - 63s 89ms/step - loss: 82.9306 - PSNR: 30.9370 - val_loss: 47.0997 - val_PSNR: 32.1734

Epoch 00079: val_PSNR did not improve from 32.41828
Epoch 80/500
700/700 [==============================] - 62s 88ms/step - loss: 149.4790 - PSNR: 29.5417 - val_loss: 77.4819 - val_PSNR: 29.7540

Epoch 00080: val_PSNR did not improve from 32.41828
Epoch 81/500
700/700 [==============================] - 54s 77ms/step - loss: 87.6688 - PSNR: 30.3704 - val_loss: 45.4573 - val_PSNR: 32.2478

Epoch 00081: val_PSNR did not improve from 32.41828
Epoch 82/500
700/700 [==============================] - 52s 74ms/step - loss: 85.2143 - PSNR: 30.7639 - val_loss: 45.5403 - val_PSNR: 32.2538

Epoch 00082: val_PSNR did not improve from 32.41828
Epoch 83/500
700/700 [==============================] - 60s 85ms/step - loss: 87.9834 - PSNR: 30.6321 - val_loss: 44.4182 - val_PSNR: 32.4851

Epoch 00083: val_PSNR improved from 32.41828 to 32.48508, saving model to weights/FSRCNN3

...


Epoch 00238: val_PSNR did not improve from 33.45497
Epoch 239/500
700/700 [==============================] - 39s 55ms/step - loss: 71.1994 - PSNR: 32.0466 - val_loss: 35.3636 - val_PSNR: 33.4538

Epoch 00239: val_PSNR did not improve from 33.45497
Epoch 240/500
700/700 [==============================] - 41s 59ms/step - loss: 73.0100 - PSNR: 31.9256 - val_loss: 35.4390 - val_PSNR: 33.4408

Epoch 00240: val_PSNR did not improve from 33.45497
Epoch 241/500
700/700 [==============================] - 42s 59ms/step - loss: 68.7836 - PSNR: 32.1808 - val_loss: 35.4400 - val_PSNR: 33.4410

Epoch 00241: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 00241: val_PSNR did not improve from 33.45497
Epoch 242/500
700/700 [==============================] - 49s 70ms/step - loss: 72.5117 - PSNR: 32.0253 - val_loss: 35.2303 - val_PSNR: 33.4738

Epoch 00242: val_PSNR improved from 33.45497 to 33.47384, saving model to weights/FSRCNN3.hdf5
Epoch 243/500
700/700 [================

700/700 [==============================] - 39s 55ms/step - loss: 72.6296 - PSNR: 32.0677 - val_loss: 35.1006 - val_PSNR: 33.4978

Epoch 00278: val_PSNR improved from 33.49769 to 33.49778, saving model to weights/FSRCNN3.hdf5
Epoch 279/500
700/700 [==============================] - 46s 66ms/step - loss: 70.2857 - PSNR: 32.0866 - val_loss: 35.1203 - val_PSNR: 33.4961

Epoch 00279: val_PSNR did not improve from 33.49778
Epoch 280/500
700/700 [==============================] - 52s 75ms/step - loss: 69.7456 - PSNR: 32.0972 - val_loss: 35.1162 - val_PSNR: 33.4961

Epoch 00280: val_PSNR did not improve from 33.49778
Epoch 281/500
700/700 [==============================] - 43s 62ms/step - loss: 69.3221 - PSNR: 32.1269 - val_loss: 35.1175 - val_PSNR: 33.4976

Epoch 00281: val_PSNR did not improve from 33.49778
Epoch 282/500
700/700 [==============================] - 38s 54ms/step - loss: 71.7935 - PSNR: 32.0916 - val_loss: 35.1091 - val_PSNR: 33.4985

Epoch 00282: val_PSNR improved from 33.4977

700/700 [==============================] - 36s 52ms/step - loss: 72.1244 - PSNR: 32.0130 - val_loss: 35.1109 - val_PSNR: 33.5007

Epoch 00317: val_PSNR did not improve from 33.50129
Epoch 318/500
700/700 [==============================] - 41s 59ms/step - loss: 70.0878 - PSNR: 32.0498 - val_loss: 35.1106 - val_PSNR: 33.4995

Epoch 00318: val_PSNR did not improve from 33.50129
Epoch 319/500
700/700 [==============================] - 36s 52ms/step - loss: 71.0148 - PSNR: 32.0828 - val_loss: 35.1034 - val_PSNR: 33.5009

Epoch 00319: val_PSNR did not improve from 33.50129
Epoch 320/500
700/700 [==============================] - 42s 59ms/step - loss: 70.4583 - PSNR: 32.1405 - val_loss: 35.1035 - val_PSNR: 33.5011

Epoch 00320: val_PSNR did not improve from 33.50129
Epoch 321/500
700/700 [==============================] - 37s 52ms/step - loss: 72.2680 - PSNR: 31.9523 - val_loss: 35.1069 - val_PSNR: 33.5000

Epoch 00321: val_PSNR did not improve from 33.50129
Epoch 322/500
700/700 [===========


Epoch 00356: val_PSNR did not improve from 33.50191
Epoch 357/500
700/700 [==============================] - 33s 47ms/step - loss: 71.3680 - PSNR: 31.9795 - val_loss: 35.1035 - val_PSNR: 33.5011

Epoch 00357: val_PSNR did not improve from 33.50191
Epoch 358/500
700/700 [==============================] - 42s 60ms/step - loss: 70.5548 - PSNR: 32.0906 - val_loss: 35.1037 - val_PSNR: 33.5012

Epoch 00358: val_PSNR did not improve from 33.50191
Epoch 359/500
700/700 [==============================] - 35s 51ms/step - loss: 71.3059 - PSNR: 32.0621 - val_loss: 35.1033 - val_PSNR: 33.5009

Epoch 00359: val_PSNR did not improve from 33.50191
Epoch 360/500
700/700 [==============================] - 38s 54ms/step - loss: 68.7296 - PSNR: 32.1329 - val_loss: 35.1034 - val_PSNR: 33.5011

Epoch 00360: val_PSNR did not improve from 33.50191
Epoch 361/500
700/700 [==============================] - 40s 57ms/step - loss: 69.2369 - PSNR: 32.1279 - val_loss: 35.1023 - val_PSNR: 33.5011

Epoch 00361: val_PS

KeyboardInterrupt: 